In [1]:
import sparknlp
from pyspark.ml import PipelineModel, Pipeline
import sparknlp.annotator as sa
import sparknlp.base as sb
import sparknlp
from sparknlp import Finisher

# Load Spark

In [2]:
from pyspark.sql import SparkSession
# spark = sparknlp.start()

spark = (
    SparkSession.builder
      .config("spark.executor.instances", "30")
      .config("spark.driver.memory", "20g")
      .config("spark.executor.memory", "20g")
      .config("spark.kryoserializer.buffer.max", "2000M")
      .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.7.5")
      .getOrCreate()
)

In [3]:
spark

# Load Our Own Data

In [4]:
import sqlite3
import pandas as pd
from tqdm.auto import tqdm
import pyspark.sql.functions as F
# import unidecode

# conn = sqlite3.connect('../data/diffengine-diffs/db/newssniffer-nytimes.db')
conn = sqlite3.connect('newssniffer-nytimes.db')

df = pd.read_sql('''
     SELECT * from entryversion 
     WHERE entry_id IN (SELECT distinct entry_id FROM entryversion LIMIT 200)
 ''', con=conn)

# df = pd.read_sql('''
#     SELECT entry_id, summary, version from entryversion 
# ''', con=conn)

df = df.assign(summary=lambda df: df['summary'].str.replace('</p><p>', ' '))

In [5]:
sdf = spark.createDataFrame(df)

# Try Sentence Tokenizing on Our Own Data

In [ ]:
documenter = sb.DocumentAssembler()\
    .setInputCol("summary")\
    .setOutputCol("document")

sentencer = (sa.SentenceDetector()
                .setInputCols(["document"])
                .setOutputCol("sentences")            
            )

finisher = (
    Finisher()
    .setInputCols(["sentences"]) 
)

sd_pipeline = PipelineModel(stages=[documenter, sentencer, finisher])

In [ ]:
annotations_df = sd_pipeline.transform(sdf)

In [ ]:
sent_list_df = (annotations_df
                .select("entry_id", "version", F.posexplode("finished_sentences"))
                .withColumnRenamed('col', 'sentence')
                .withColumnRenamed('pos', 'sent_idx')
               )
# tdf = sent_list_df.toPandas()

In [54]:
exploded_sent_df = (sent_list_df
 .alias("sent_list_df")
 .join(
     sent_list_df.alias("sent_list_df_2"),
     [F.col("sent_list_df.entry_id") == F.col("sent_list_df_2.entry_id"), 
      F.col("sent_list_df.version") == F.col("sent_list_df_2.version"), 
     ], 
     "inner"
 )
 .select(
     F.col("sent_list_df.entry_id"),
     F.col("sent_list_df.version"),
     F.col("sent_list_df.sent_idx").alias("sent_idx_x"),
     F.col("sent_list_df_2.sent_idx").alias("sent_idx_y"),
     F.col("sent_list_df.sentence").alias("sentence_x"),
     F.col("sent_list_df_2.sentence").alias("sentence_y"),
#    .show(truncate=False)
    )
)

In [55]:
exploded_sent_df.show()


## todo: 
## 0. do this same procedure for diffed sequential versions

## 1a. use tokenize and Albert or BERT or Word2Vec to generate vectors of embeddings for each sentence.
## 1b. lemmatize each sentence

## 2. take Sim_asym along each row, two times using:
## a. phi(x, y) = vec(x) \cdot vec(y)
## b. phi(x ,y) = lemmatization

## 3. for each sentence, select the argmax in both directions.
## 4. choose some reasonable threshold.

## 5. For scores above this threshold, co

+--------+-------+----------+----------+-------------------+--------------------+
|entry_id|version|sent_idx_x|sent_idx_y|         sentence_x|          sentence_y|
+--------+-------+----------+----------+-------------------+--------------------+
|  548743|      1|         0|         0|FORT COLLINS, Colo.| FORT COLLINS, Colo.|
|  548743|      1|         0|         1|FORT COLLINS, Colo.|— Annie Hartnett ...|
|  548743|      1|         0|         2|FORT COLLINS, Colo.|Now 21 and a lead...|
|  548743|      1|         0|         3|FORT COLLINS, Colo.|“I would still sa...|
|  548743|      1|         0|         4|FORT COLLINS, Colo.|“When you’re voti...|
|  548743|      1|         0|         5|FORT COLLINS, Colo.|” So on Saturday ...|
|  548743|      1|         0|         6|FORT COLLINS, Colo.|Each party used t...|
|  548743|      1|         0|         7|FORT COLLINS, Colo.|But Mr. Obama, tr...|
|  548743|      1|         0|         8|FORT COLLINS, Colo.|“I’m counting on ...|
|  548743|      

In [32]:
tdf

,entry_id,version,pos,col
0,547988,0,0,"In Silicon Valley, Apple just won big against ..."
1,547988,0,1,"Across the country, in a federal court in Flor..."
2,547988,0,2,"Mr. Stadnyk, who holds a patent on a motorcycl..."
3,547988,0,3,"Represented by a prominent Washington lawyer, ..."
4,547988,0,4,Mr. Stadnyk and his lawyer — along with some a...
5,547988,0,5,"The present system, one of the nation’s oldest..."
6,547988,0,6,The impending law would overturn that by award...
7,547988,0,7,"Mr. Stadnyk, 48, a garage inventor who stumble..."
8,547988,0,8,He devised a system of brackets and gears to a...
9,547988,0,9,"With his system, he says, the rider feels a fl..."


In [ ]:
chunksize = 10000
unique_entryids = df['entry_id'].unique()
num_chunks = int(unique_entryids.shape[0] / chunksize)

output_dfs = []
for chunk_id in tqdm(range(num_chunks)):
    batch_ids = unique_entryids[chunk_id * chunksize: (chunk_id + 1) * chunksize]
    small_df = df.loc[lambda df: df['entry_id'].isin(batch_ids)]
    #
    sdf = spark.createDataFrame(small_df)
    #
    annotations_df = sd_pipeline.transform(sdf)
    t_df = annotations_df.toPandas()
    output_dfs.append(t_df)

# Get Albert Embeddings

In [5]:
document_assembler = (
      sb.DocumentAssembler()
        .setInputCol("summary")
        .setOutputCol("document")
)

tokenizer = (
    sa.Tokenizer()
        .setInputCols(["document"])
        .setOutputCol("token")
)
 
word_embeddings = (
    sa.AlbertEmbeddings
        .load('s3://aspangher/spark-nlp/albert_xxlarge_uncased_en')
        .setInputCols(["document", "token"])
        .setOutputCol("embeddings")
)

embeddings_finisher = (
    sb.EmbeddingsFinisher()
            .setInputCols("embeddings")
            .setOutputCols("embeddings_vectors")
            .setOutputAsVector(True)
)

In [37]:
tokenizer = (
    sa.Tokenizer()
        .setInputCols(["document"])
        .setOutputCol("token")
)

In [38]:
bert_pipeline = Pipeline(stages=
  [
    document_assembler,
    tokenizer,
    word_embeddings,
    embeddings_finisher
  ]
)

In [40]:
df_bert = bert_pipeline.fit(sdf).transform(sdf)
# df_bert = bert_pipeline_model.transform(sdf)

In [42]:
df_bert#.select('entry_id', 'version', 'embedding_vectors')

DataFrame[index: bigint, version: bigint, title: string, created: string, url: string, source: string, entry_id: bigint, archive_url: string, num_versions: bigint, summary: string, joint_key: string, id: string, document: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, token: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, embeddings: array<struct<annotatorType:string,begin:int,end:int,result:string,metadata:map<string,string>,embeddings:array<float>>>, embeddings_vectors: array<vector>]

In [ ]:
t2_df = (df_bert
         .select('entry_id', 'version', 'embeddings_vectors')
         .toPandas()
        )

# With Sentences

In [16]:
documenter = (
    sb.DocumentAssembler()
        .setInputCol("summary")
        .setOutputCol("document")
)

sentencer = (
    sa.SentenceDetector()
        .setInputCols(["document"])
        .setOutputCol("sentences")            
)

tokenizer = (
    sa.Tokenizer()
        .setInputCols(["sentences"])
        .setOutputCol("token")
)

word_embeddings = (
    sa.AlbertEmbeddings
        .load('s3://aspangher/spark-nlp/albert_large_uncased_en')
        .setInputCols(["sentences", "token"])
        .setOutputCol("embeddings")
        .setBatchSize(100)
)

In [17]:
embeddings_finisher = (
    sb.EmbeddingsFinisher()
            .setInputCols("embeddings")
            .setOutputCols("embeddings_vectors")
            .setOutputAsVector(True)
)

In [18]:
bert_pipeline_from_sentences = Pipeline(stages=
  [
      documenter,
#       sentence_documenter,
      sentencer,
      tokenizer,
      word_embeddings,
      embeddings_finisher
  ]
)

In [19]:
# df_bert = bert_pipeline_from_sentences.fit(sent_list_df).transform(sent_list_df)
df_bert = bert_pipeline_from_sentences.fit(sdf).transform(sdf)

In [13]:
# df_bert.show()

In [21]:
spark

In [22]:
(df_bert
 .select('entry_id', 'version', 'sentences', 'embeddings_vectors')
 .write.mode("overwrite").parquet("s3://aspangher/tmp/tmp_albert_embeddings"))

Py4JJavaError: An error occurred while calling o617.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

In [ ]:
df_bert.select('entry_id', 'version', 'embeddings_vectors').show()

In [39]:
pipeline_model = bert_pipeline_from_sentences.fit(spark.createDataFrame([[""]]).toDF("summary"))
result = pipeline_model.transform(spark.createDataFrame(pd.DataFrame({"summary": ["I love NLP"]})))

In [ ]:
result.show()

In [ ]:
t2_df = (df_bert
         .select('entry_id', 'version', 'sent_idx', 'sentence', 'embeddings_vectors')
         .toPandas()
        )

# Test Data

In [ ]:
# test_data = CoNLL().readDataset(spark, 's3://aspangher/spark-nlp/conll/eng.train')

In [ ]:
from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, 's3://aspangher/spark-nlp/conll/eng.train')

get_embeddings = (sa.AlbertEmbeddings
        .load('s3://aspangher/spark-nlp/albert_large_uncased_en')
        .setInputCols("sentence", "token")
        .setOutputCol("embeddings")
        .setMaxSentenceLength(100)
        .setBatchSize(8)
)

embeddings_finisher = (
    sb.EmbeddingsFinisher()
        .setInputCols("embeddings")
        .setOutputCols("embeddings_vectors")
        .setOutputAsVector(True)
)

sentence_finisher = (
    Finisher()
       .setInputCols(["sentence"]) 
)

pipeline =  Pipeline(stages=[
    get_embeddings, 
    embeddings_finisher, 
    sentence_finisher
])

pipelineDF = pipeline.fit(training_data).transform(training_data)

(pipelineDF
 .select('finished_sentence', 'embeddings_vectors')
 .write
 .mode("overwrite").parquet('s3://aspangher/tmp/tmp_conll_albert_embeddings.pq')
)

In [29]:
sparknlp.version()

'2.7.5'

In [30]:
spark.version

'2.4.4'

In [ ]:
pipelineDF.select('finished_sentence', 'embeddings_vectors').show()

In [ ]:
ner_pipeline = RecursivePipeline(stages=[
    DocumentAssembler().setInputCol("text").setOutputCol("document"), 
    SentenceDetector().setInputCols(["document"]).setOutputCol("sentence"), 
    Tokenizer().setInputCols(["sentence"]).setOutputCol("token").setMaxLength(100).setSplitChars(["-", "\xa0", "—"]), 
    BertEmbeddings.pretrained(name = "bert_large_cased", lang='en').setInputCols(['sentence', 'token']).setOutputCol('embeddings'), 
#     NerDLModel.pretrained('onto_bert_large_cased', 'en').setInputCols(['sentence', 'token', 'embeddings']).setOutputCol('ner'), 
#     NerConverter().setInputCols(['sentence', 'token', 'ner']).setOutputCol('ner_chunk') 
])